# Instacart Feature Engineering

In [33]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random
import datetime

#deep learning 
import tensorflow as tf

#sklearn 
from sklearn.preprocessing import MinMaxScaler

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Read and merge data

In [4]:
#read data
order_products_prior_df = pd.read_csv('./data/order_products__prior.csv', engine='c',
                                        dtype={'order_id': np.int32, 'product_id': np.int32,
                                        'add_to_cart_order': np.int16, 'reordered': np.int8})
print('shape of order_products_prior_df is {}'.format(order_products_prior_df.shape))

order_products_train_df = pd.read_csv('./data/order_products__train.csv', engine='c',
                                      dtype={'order_id': np.int32, 'product_id': np.int32,
                                             'add_to_cart_order': np.int16, 'reordered': np.int8})
print('shape of order_products_train_df is {}'.format(order_products_train_df.shape))

orders_df = pd.read_csv('./data/orders.csv', engine='c',
                        dtype={'order_id': np.int32, 'user_id': np.int32, 'order_number': np.int32,
                               'order_dow': np.int8, 'order_hour_of_day': np.int8,
                               'days_since_prior_order': np.float16})
print('shape of orders_df is {}'.format(orders_df.shape))

products_df = pd.read_csv("./data/products.csv", engine='c')
print('shape of products_df is {}'.format(products_df.shape))

aisles = pd.read_csv("./data/aisles.csv")
print('shape of aisles is {}'.format(aisles.shape))

departments = pd.read_csv("./data/departments.csv")
print('shape of departments is {}'.format(departments.shape))

#merge data
df_products_orders_all = pd.concat([order_products_prior_df, order_products_train_df])

print('merging order_products with orders')

df_products_orders_all = pd.merge(df_products_orders_all, orders_df, on='order_id')
print('shape of df_products_orders_all is {}'.format(df_products_orders_all.shape))

print('grabbing aisles and departments names ')

df = df_products_orders_all.merge(products_df, how='inner', on='product_id')
df = df.merge(aisles, how='inner', on='aisle_id')
df = df.merge(departments, how='inner', on='department_id')
print('shape of df is {}'.format(df.shape))

print('sorting by user and order...')

df = df.drop(columns=['aisle_id','department_id','product_name'], axis=1)
df.sort_values(['user_id', 'order_number', 'add_to_cart_order'], axis=0, inplace=True)
df = df.reset_index(drop=True)
df.head()

shape of order_products_prior_df is (32434489, 4)
shape of order_products_train_df is (1384617, 4)
shape of orders_df is (3421083, 7)
shape of products_df is (49688, 4)
shape of aisles is (134, 2)
shape of departments is (21, 2)
merging order_products with orders
shape of df_products_orders_all is (33819106, 10)
grabbing aisles and departments names 
shape of df is (33819106, 15)
sorting by user and order...


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
0,2539329,196,1,0,1,prior,1,2,8,NaN,soft drinks,beverages
1,2539329,14084,2,0,1,prior,1,2,8,NaN,soy lactosefree,dairy eggs
2,2539329,12427,3,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
3,2539329,26088,4,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
4,2539329,26405,5,0,1,prior,1,2,8,NaN,paper goods,household


In [5]:
df_sample = pd.read_csv('./data/df_sample_max_7_orders.csv')
print(df_sample.shape)
df_sample.head()

(13817856, 12)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
0,2539329,196,1,0,1,prior,1,2,8,NaN,soft drinks,beverages
1,2539329,14084,2,0,1,prior,1,2,8,NaN,soy lactosefree,dairy eggs
2,2539329,12427,3,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
3,2539329,26088,4,0,1,prior,1,2,8,NaN,popcorn jerky,snacks
4,2539329,26405,5,0,1,prior,1,2,8,NaN,paper goods,household


In [8]:
debug=True

if debug:
    df = df_sample
    print(df.shape)

(13817856, 12)


## Feature engineering

In [11]:
user_order_basket_size=pd.DataFrame(df.groupby(['user_id','order_id']).size()).rename(columns={0:'basket_size'}).reset_index()
basket_size = user_order_basket_size.merge(df, how='left', on=['user_id','order_id'])['basket_size']

In [12]:
# count of products, aisles and departments per order
basket_stats = (df
                .groupby(['user_id', 'order_id'])
                .agg({'product_id':'count','department':'nunique', 'aisle':'nunique'})
                .reset_index()
                .rename(columns={'product_id':'count_products',
                                 'department':'count_departments'
                                 ,'aisle':'count_aisles'}))

df = df.merge(basket_stats, how='left', on=['user_id','order_id'])

gc.collect()

850

In [13]:
# count of orders per user
orders_per_user = df.groupby('user_id').agg({'order_id':'nunique'}).reset_index().rename(columns={'order_id':'total_orders_per_user'})
df = df.merge(orders_per_user, how='left', on=['user_id'])

del orders_per_user
gc.collect()

df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user
0,2539329,196,1,0,1,prior,1,2,8,NaN,soft drinks,beverages,5,4,4,8
1,2539329,14084,2,0,1,prior,1,2,8,NaN,soy lactosefree,dairy eggs,5,4,4,8
2,2539329,12427,3,0,1,prior,1,2,8,NaN,popcorn jerky,snacks,5,4,4,8
3,2539329,26088,4,0,1,prior,1,2,8,NaN,popcorn jerky,snacks,5,4,4,8
4,2539329,26405,5,0,1,prior,1,2,8,NaN,paper goods,household,5,4,4,8


In [14]:
# cumsum of user lifetime
df.days_since_prior_order = df.days_since_prior_order.fillna(0)
cum_days_since_prior_order = df.groupby(['user_id','order_id']).agg({'days_since_prior_order':'mean'}).reset_index()
cum_days_since_prior_order = cum_days_since_prior_order.sort_values(by=['user_id','days_since_prior_order'])
cum_days_since_prior_order['cum_days_since_prior_order'] = cum_days_since_prior_order.groupby(['user_id'])['days_since_prior_order'].cumsum()

df = df.merge(cum_days_since_prior_order[['user_id','order_id','cum_days_since_prior_order']], how='left', on=['user_id','order_id'])


# user lifetime (how long have they been ordering)
customer_lifetime = (cum_days_since_prior_order
                        .groupby('user_id')
                        .agg({'cum_days_since_prior_order': 'max'})
                        .rename(columns={'cum_days_since_prior_order':'max_cum_days_since_prior_order'})
                        .reset_index())

df = df.merge(customer_lifetime, how='left', on=['user_id'])

del cum_days_since_prior_order
del customer_lifetime
gc.collect()

df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,8,0.0,131.0
1,2539329,14084,2,0,1,prior,1,2,8,0.0,soy lactosefree,dairy eggs,5,4,4,8,0.0,131.0
2,2539329,12427,3,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,8,0.0,131.0
3,2539329,26088,4,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,8,0.0,131.0
4,2539329,26405,5,0,1,prior,1,2,8,0.0,paper goods,household,5,4,4,8,0.0,131.0


In [15]:
products_dow_stats = df.groupby(['user_id','order_dow']).agg({'order_dow':'count'}).rename(columns={'order_dow':'count_products_ordered_dow'}).reset_index()
#products_hod_stats = df.groupby(['user_id','order_hour_of_day']).agg({'order_hour_of_day':'count'}).rename(columns={'order_hour_of_day':'count_products_ordered_hod'}).reset_index()

df = df.merge(products_dow_stats, how='left', on=['user_id','order_dow'])
#df = df.merge(df, how='left', on=['user_id','order_hour_of_day'])

del products_dow_stats
gc.collect()

49

In [18]:
# user_id order hour of day stats
orders_hod_stats = df.groupby(['user_id', 'order_hour_of_day']).agg({'order_hour_of_day':['count','mean']})

# user_id order day of week stats
orders_dow_stats = df.groupby(['user_id', 'order_dow']).agg({'order_dow':['count','mean']})

# user_id basket_stats
user_id_basket_stats = (basket_stats
                            .groupby('user_id')
                            .agg({'count_products':['sum','mean','max','min'],
                                'count_departments':['mean','max','min'], 
                                'count_aisles':['mean','max','min']}))

#Number of items reordered per user
products_per_user = (df.groupby('user_id').agg({'reordered':['sum', 'mean']}))

#feature engineering from orders dataset
days_between_orders = (orders_df
                           .groupby('user_id')
                           .agg({'days_since_prior_order': ['min','max','mean']}))

# add_to_cart_order stats (mean, min, max)
product_cart_order = df.groupby(['user_id','product_id']).agg({'add_to_cart_order': ['min','max','mean']})

gc.collect()

276

In [19]:
list_groupby_features = [orders_hod_stats, orders_dow_stats, 
                         user_id_basket_stats, products_per_user, 
                         days_between_orders, product_cart_order]

for groupby_df in list_groupby_features:
    groupby_df.columns = pd.Index([e[0] + "_" + e[1].upper() for e in groupby_df.columns.tolist()])

gc.collect()

22

In [20]:
df = df.merge(orders_hod_stats, on=['user_id', 'order_hour_of_day'])
df = df.merge(orders_dow_stats, on=['user_id', 'order_dow'])
df = df.merge(user_id_basket_stats, on='user_id')
df = df.merge(products_per_user, on='user_id')
df = df.merge(days_between_orders, on='user_id')

del orders_hod_stats
del orders_dow_stats
del user_id_basket_stats
del products_per_user
del days_between_orders
gc.collect()

21

In [21]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order,count_products_ordered_dow,order_hour_of_day_COUNT,order_hour_of_day_MEAN,order_dow_COUNT,order_dow_MEAN,count_products_SUM,count_products_MEAN,count_products_MAX,count_products_MIN,count_departments_MEAN,count_departments_MAX,count_departments_MIN,count_aisles_MEAN,count_aisles_MAX,count_aisles_MIN,reordered_SUM,reordered_MEAN,days_since_prior_order_MIN,days_since_prior_order_MAX,days_since_prior_order_MEAN
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0
1,2539329,14084,2,0,1,prior,1,2,8,0.0,soy lactosefree,dairy eggs,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0
2,2539329,12427,3,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0
3,2539329,26088,4,0,1,prior,1,2,8,0.0,popcorn jerky,snacks,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0
4,2539329,26405,5,0,1,prior,1,2,8,0.0,paper goods,household,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0


In [22]:
user_product_reorders = df.groupby(['user_id','product_id']).agg({'reordered':'sum'}).rename(columns={'reordered':'sum_product_reorders_by_user'}).reset_index()
product_reorder = df.groupby('product_id').agg({'reordered':'sum'}).rename(columns={'reordered':'sum_total_product_reorders'}).reset_index()

In [23]:
df = df.merge(user_product_reorders, on=['user_id','product_id'])
df = df.merge(product_reorder, on=['product_id'])

del user_product_reorders
del product_reorder
gc.collect()

35

In [24]:
#reordered_MEAN == user_reorder_rate
df = (df
      .rename(columns = {'reordered_SUM':'sum_reordered_products_by_user',
                        'reordered_MEAN':'user_reorder_probability'})
      .assign(
        user_product_reorder_probability = lambda df: df.sum_product_reorders_by_user/df.sum_reordered_products_by_user,
        product_reorder_probability = lambda df: df.sum_total_product_reorders/df.shape[0]
    ))

gc.collect()
      
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,count_products,count_departments,count_aisles,total_orders_per_user,cum_days_since_prior_order,max_cum_days_since_prior_order,count_products_ordered_dow,order_hour_of_day_COUNT,order_hour_of_day_MEAN,order_dow_COUNT,order_dow_MEAN,count_products_SUM,count_products_MEAN,count_products_MAX,count_products_MIN,count_departments_MEAN,count_departments_MAX,count_departments_MIN,count_aisles_MEAN,count_aisles_MAX,count_aisles_MIN,sum_reordered_products_by_user,user_reorder_probability,days_since_prior_order_MIN,days_since_prior_order_MAX,days_since_prior_order_MEAN,sum_product_reorders_by_user,sum_total_product_reorders,user_product_reorder_probability,product_reorder_probability
0,2539329,196,1,0,1,prior,1,2,8,0.0,soft drinks,beverages,5,4,4,8,0.0,131.0,9,16,8,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0,7,10662,0.184211,0.000772
1,3367565,196,1,1,1,prior,6,2,7,19.0,soft drinks,beverages,4,3,4,8,33.0,131.0,9,9,7,9,2,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0,7,10662,0.184211,0.000772
2,1187899,196,1,1,1,train,11,4,8,14.0,soft drinks,beverages,11,5,10,8,14.0,131.0,24,16,8,24,4,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0,7,10662,0.184211,0.000772
3,2254736,196,1,1,1,prior,4,4,7,29.0,soft drinks,beverages,5,4,5,8,131.0,131.0,24,9,7,24,4,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0,7,10662,0.184211,0.000772
4,431534,196,1,1,1,prior,5,4,15,28.0,soft drinks,beverages,8,4,5,8,102.0,131.0,24,8,15,24,4,49,6.125,11,4,3.875,5,3,5.375,10,4,38,0.77551,0.0,30.0,19.0,7,10662,0.184211,0.000772


In [27]:
features_to_scale = ['add_to_cart_order','reordered','order_number','order_dow','order_hour_of_day','days_since_prior_order',
                     'count_products','count_departments','count_aisles','total_orders_per_user','cum_days_since_prior_order',
                     'max_cum_days_since_prior_order','count_products_ordered_dow','order_hour_of_day_COUNT','order_hour_of_day_MEAN',
                     'order_dow_COUNT','order_dow_MEAN','count_products_SUM','count_products_MEAN','count_products_MAX',
                     'count_products_MIN','count_departments_MEAN','count_departments_MAX','count_departments_MIN','count_aisles_MEAN',
                     'count_aisles_MAX','count_aisles_MIN','sum_reordered_products_by_user','user_reorder_probability',
                     'days_since_prior_order_MIN','days_since_prior_order_MAX','days_since_prior_order_MEAN','sum_product_reorders_by_user',
                     'sum_total_product_reorders','user_product_reorder_probability','product_reorder_probability']

In [26]:
#get dummies
df = pd.get_dummies(df, prefix=['aisle','department'], columns=['aisle','department'])
gc.collect()

In [ ]:
#scale features
df[features_to_scale] = (df[features_to_scale]-df[features_to_scale].min())/((df[features_to_scale].max()-df[features_to_scale].min()))

gc.collect()
df.head()

In [ ]:
gc.collect()